In [ ]:
import pandas as pd
import torch
import pickle
from argparse import Namespace
import re
import sys
import os
print('current path: ',os.getcwd())
current_path = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)


from utils.utilities_DL import get_loss,load_model_and_optimizer
from build_inputs.load_datasets_to_predict import load_datasets_to_predict
from calendar_class import get_time_slots_labels

trial_ids=['netmob_subway_in_calendar_STGCN_FeatureExtractor_ResNetInspired_MSELoss_2024_08_28_06_04_41108',
                 'netmob_subway_in_calendar_STGCN_ImageAvgPooling_MSELoss_2024_08_27_00_16_90667',
                 'netmob_subway_in_STGCN_FeatureExtractor_ResNetInspired_MSELoss_2024_08_23_06_53_46982',
                 'netmob_subway_in_STGCN_ImageAvgPooling_MSELoss_2024_08_24_01_42_17375',
                 'subway_in_calendar_STGCN_MSELoss_2024_08_25_22_56_92429',
                 'subway_in_STGCN_MSELoss_2024_08_25_18_05_25229']

save_path = 'save/K_fold_validation/training_with_HP_tuning/'

current path:  /home/rrochas/prediction-validation/jupyter_ipynb/Entropy_and_information/examples


## Find all the fold save associated to a HP-tuning trial 

In [48]:
model_args = pickle.load(open(f'{working_dir}/{save_path}/best_models/model_args.pkl','rb'))
#trial_id = 'subway_in_STGCN_MSELoss_2024_08_25_18_05_25229'
trial_id =  'subway_in_subway_out_ASTGCN_MSELoss_2025_04_21_20_06_76371'
best_model_names = [name for name in model_args['model'].keys() if trial_id in name]

re.pattern = re.compile(trial_id)
csv_trial_id = [name for name in os.listdir(f'{working_dir}/{save_path}') if re.search(trial_id, name)]
print('CSV with saved metrics:')
display(csv_trial_id)

CSV with saved metrics:


['VALID_subway_in_subway_out_ASTGCN_MSELoss_2025_04_21_20_06_76371.csv',
 'Losses_subway_in_subway_out_ASTGCN_MSELoss_2025_04_21_20_06_76371.csv',
 'METRICS_subway_in_subway_out_ASTGCN_MSELoss_2025_04_21_20_06_76371.csv',
 'METRICS_BY_FOLDsubway_in_subway_out_ASTGCN_MSELoss_2025_04_21_20_06_76371.csv']

## Select one best model: 

In [56]:
selected_model_name = best_model_names[0]
args = model_args['model'][selected_model_name]['args']
perf = model_args['model'][selected_model_name]['performance']
losses_path = [csv_path for csv_path in csv_trial_id if 'Losses' in csv_path][0]
losses = pd.read_csv(f'{working_dir}/{save_path}{losses_path}',index_col=0)
args = Namespace(**args)

## Print its associated best Metrics: 

In [57]:
import numpy as np 
valid_mae = []
valid_mse = []
test_mae = []
test_mse = []

for model_name in best_model_names:
    valid_mae.append(model_args['model'][model_name]['performance']['valid_metrics']['mae'])
    test_mae.append(model_args['model'][model_name]['performance']['test_metrics']['mae'])
    valid_mse.append(model_args['model'][model_name]['performance']['valid_metrics']['mse'])
    test_mse.append(model_args['model'][model_name]['performance']['test_metrics']['mse'])


df_losses = pd.DataFrame([valid_mae,valid_mse,test_mae,test_mse],columns = [f"fold_{k}" for k in range(len(valid_losses))],index = ['Valid MAE','Valid MSE','Test MAE','Test MSE'])
display(df_losses)

,fold_0,fold_1,fold_2,fold_3,fold_4
Valid MAE,22.568340,21.601465,21.345070,20.062286,20.997963
Valid MSE,1469.476318,1398.276123,1284.733887,1216.252808,1278.792114
Test MAE,20.981377,22.612219,21.797729,21.479622,21.652296
Test MSE,1297.973145,1426.760010,1377.152710,1319.273071,1378.403809


## Print Losses through epochs: 

In [59]:
from plotting.TS_analysis import plot_TS
plot_TS(losses,width=1200,height=400,bool_show=True,title=f"Losses through epochs",x_datetime = False)

Loading BokehJS ...

figure(id='p1577', ...)

## Load the model and its weights:

In [ ]:
from constants.paths import FILE_NAME
from utils.utilities_DL match_period_coverage_with_netmob

selected_model_path = f"{working_dir}/{save_path}/{selected_model_name}.pkl"
model_param = torch.load(selected_model_path)

# Load Model:
coverage_period = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])
dataset,_,_ = load_datasets_to_predict(args,coverage_period)
_,dic_class2rpz,_,_ = get_time_slots_labels(dataset,nb_class = [0,1,2,3])
loss_function = get_loss(args)
model,optimizer,scheduler = load_model_and_optimizer(args,dic_class2rpz)

# Associate its weights: 
model.load_state_dict(model_param['state_dict'])

Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00

Init Subway-In Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 


 Tackling Training Set

 Tackling Validation Set

 Tackling Testing Set

 Tackling Training Set

 Tackling Validation Set

 Tackling Testing Set

U size:  torch.Size([6719, 40, 7]) Utarget size:  torch.Size([6719, 40, 1])
U_train size:  torch.Size([3912, 40, 7]) Utarget_train size:  torch.Size([3912, 40, 1])
U_valid size:  torch.Size([1304, 40, 7]) Utarget_valid size:  torch.Size([1304, 40, 1])
U_test size:  torch.Size([1310, 40, 7]) Utarget_test size:  torch.Size([1310, 40, 1])
U_train min:  tensor(0.) U_train max:  tensor(1.)
U_valid min:  tensor(0.) U_valid max:  tensor(1.7234)
U_test min:  tensor(0.) U_test max:  tensor(2.2126)
model size: 0.001GB
number of total parameters: 249889
number of trainable parameters: 249889


<All keys matched successfully>